In [1]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.core.metrics import make_scorer


#Basic
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score

# #loss function metrics
# from scipy.stats import lognorm, loguniform, randint, uniform
# from sklearn.metrics import (accuracy_score, classification_report,
#                              confusion_matrix, f1_score, make_scorer,
#                              precision_score, recall_score,
#                              average_precision_score, auc,
#                             roc_auc_score,roc_curve,precision_recall_curve,
#                             PrecisionRecallDisplay,ConfusionMatrixDisplay)

# #model_selection and hyperparameter tunning
# from sklearn.model_selection import (GridSearchCV, RandomizedSearchCV,
#                                      cross_val_score, cross_validate,
#                                      train_test_split, cross_val_predict)

# #pipeline
# from sklearn.pipeline import Pipeline, make_pipeline
# from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
# from sklearn.compose import ColumnTransformer, make_column_transformer

# #Classifiers
# from catboost import CatBoostClassifier
# from lightgbm.sklearn import LGBMClassifier
# from sklearn.tree import DecisionTreeClassifier
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression

# #Imbalanced-Learn Library
# #from imblearn.over_sampling import SMOTE
# #from imblearn.pipeline import make_pipeline as make_imb_pipeline

# #
# from sklearn.ensemble import BaggingClassifier

# #Explain predictions
# # import eli5
# # import shap

# #For Kendall Correlation
# import scipy.stats as stats

import os
pd.set_option('display.max_columns', 500)

print(os.getcwd())

/mnt/c/Users/diego.villacreses/OneDrive - Universidad de Las Américas/Desktop/MT/credit_default/notebooks


/home/user/miniconda3/envs/ag_gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_excel("../data/raw/base_pagares.xlsx")
df = df.drop(columns=["Unnamed: 0","CEDULA", "is_default", "cedula", "FechaDescarga", "JSON","dias_to_pago"])

# dias_to_pago predictor perfecto?

In [3]:
print("Distribution of the target variable - full data:\n",
      df["target"].value_counts(normalize=True),
      sep='')

Distribution of the target variable - full data:
target
0    0.732947
1    0.267053
Name: proportion, dtype: float64


In [4]:
def school_profit(y_true, y_pred,alpha=5,beta=10,gamma=1):
    true_positive = sum(y_pred[y_pred==y_true] ==1)
    true_negative = sum(y_pred[y_pred==y_true] ==0)
    false_positive = sum(y_pred[y_pred!=y_true] ==1)
    false_negative = sum(y_pred[y_pred!=y_true] ==0)
    
    profit = beta*true_negative - alpha * false_positive - gamma*false_negative
    #profit = beta*true_negative + beta*true_positive - alpha * false_positive - alpha * false_negative
    return profit

ag_accuracy_scorer = make_scorer(name='school_profit',
                                 score_func=school_profit,
                                 #optimum=35_000,
                                 greater_is_better=True)


f1 = make_scorer(name="f1_binary", 
score_func=f1_score, 
average = "binary",
greater_is_better=True,
optimum = 1)

In [5]:
minutes = 60
hours = 48+8

label = "target"
predictor_ag = TabularPredictor(label=label, eval_metric = f1)
predictor_ag.fit(df,
     presets='best_quality', #medium_quality, good_quality, best_quality
     time_limit = 60*minutes*hours, # in seconds
     num_gpus=1
     )

No path specified. Models will be saved in: "AutogluonModels/ag-20240223_231111"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 201600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240223_231111/ds_sub_fit/sub_fit_ho.
Beginning AutoGluon training ... Time limit = 50400s
AutoGluon will save models to "AutogluonModels/ag-20240223_231111/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:

[1000]	valid_set's binary_logloss: 0.641342	valid_set's f1_binary: 0.27957
[1000]	valid_set's binary_logloss: 0.633752	valid_set's f1_binary: 0.236111
[1000]	valid_set's binary_logloss: 0.687738	valid_set's f1_binary: 0.165468
[1000]	valid_set's binary_logloss: 0.638353	valid_set's f1_binary: 0.221374
[1000]	valid_set's binary_logloss: 0.674149	valid_set's f1_binary: 0.169742
[1000]	valid_set's binary_logloss: 0.690155	valid_set's f1_binary: 0.179856
[2000]	valid_set's binary_logloss: 0.7828	valid_set's f1_binary: 0.226804


	0.2398	 = Validation score   (f1_binary)
	31.62s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 33558.26s of the 50366.57s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's binary_logloss: 0.713961	valid_set's f1_binary: 0.264286
[2000]	valid_set's binary_logloss: 0.882468	valid_set's f1_binary: 0.264808
[1000]	valid_set's binary_logloss: 0.728438	valid_set's f1_binary: 0.26087
[1000]	valid_set's binary_logloss: 0.734588	valid_set's f1_binary: 0.207407
[1000]	valid_set's binary_logloss: 0.788877	valid_set's f1_binary: 0.181132
[1000]	valid_set's binary_logloss: 0.76484	valid_set's f1_binary: 0.19697


	0.2325	 = Validation score   (f1_binary)
	21.8s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: RandomForestGini_BAG_L1 ... Training model for up to 33535.71s of the 50344.02s of remaining time.
	0.1165	 = Validation score   (f1_binary)
	0.66s	 = Training   runtime
	0.16s	 = Validation runtime
Fitting model: RandomForestEntr_BAG_L1 ... Training model for up to 33534.38s of the 50342.69s of remaining time.
	0.1118	 = Validation score   (f1_binary)
	0.61s	 = Training   runtime
	0.16s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 33533.07s of the 50341.38s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	0.0226	 = Validation score   (f1_binary)
	12.28s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: ExtraTreesGini_BAG_L1 ... Training model for up to 33520.45s of the 50328.76s of remaining time.
	0.1269	 = Validation score   (f1_binary)
	0.61s	 = Training   run

...

	0.2494	 = Validation score   (f1_binary)
	21.72s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: XGBoost_r22_BAG_L1 ... Training model for up to 190159.66s of the 190159.66s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	0.2428	 = Validation score   (f1_binary)
	38.14s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: NeuralNetFastAI_r69_BAG_L1 ... Training model for up to 190120.86s of the 190120.86s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
Metric f1_binary is not supported by this model - using log_loss instead
Metric f1_binary is not supported by this model - using log_loss instead
Metric f1_binary is not supported by this model - using log_loss instead
Metric f1_binary is not supported by this model - using log_loss instead
Metric f1_binary is not supported by this model - using log_loss instead
Metric f1_binary is not sup

[1000]	valid_set's binary_logloss: 0.638991	valid_set's f1_binary: 0.208955
[1000]	valid_set's binary_logloss: 0.677714	valid_set's f1_binary: 0.11985
[1000]	valid_set's binary_logloss: 0.719675	valid_set's f1_binary: 0.164286
[1000]	valid_set's binary_logloss: 0.686777	valid_set's f1_binary: 0.215054
[1000]	valid_set's binary_logloss: 0.66932	valid_set's f1_binary: 0.169118


	0.1828	 = Validation score   (f1_binary)
	40.88s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetFastAI_r172_BAG_L1 ... Training model for up to 189972.16s of the 189972.15s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
Metric f1_binary is not supported by this model - using log_loss instead
No improvement since epoch 1: early stopping
Metric f1_binary is not supported by this model - using log_loss instead
No improvement since epoch 1: early stopping
Metric f1_binary is not supported by this model - using log_loss instead
No improvement since epoch 1: early stopping
Metric f1_binary is not supported by this model - using log_loss instead
No improvement since epoch 1: early stopping
Metric f1_binary is not supported by this model - using log_loss instead
No improvement since epoch 1: early stopping
Metric f1_binary is not supported by this model - using log_loss instead
No improvement since epoch 1:

[1000]	valid_set's binary_logloss: 0.62887	valid_set's f1_binary: 0.288401
[2000]	valid_set's binary_logloss: 0.690746	valid_set's f1_binary: 0.303951
[1000]	valid_set's binary_logloss: 0.647165	valid_set's f1_binary: 0.309148
[1000]	valid_set's binary_logloss: 0.653234	valid_set's f1_binary: 0.227273
[1000]	valid_set's binary_logloss: 0.634708	valid_set's f1_binary: 0.28115
[1000]	valid_set's binary_logloss: 0.67175	valid_set's f1_binary: 0.229508
[1000]	valid_set's binary_logloss: 0.690397	valid_set's f1_binary: 0.255452
[1000]	valid_set's binary_logloss: 0.699094	valid_set's f1_binary: 0.250804
[1000]	valid_set's binary_logloss: 0.660145	valid_set's f1_binary: 0.244224
[2000]	valid_set's binary_logloss: 0.728056	valid_set's f1_binary: 0.264615
[3000]	valid_set's binary_logloss: 0.798843	valid_set's f1_binary: 0.269795


	0.2776	 = Validation score   (f1_binary)
	31.9s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetTorch_r1_BAG_L1 ... Training model for up to 188977.1s of the 188977.1s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	0.3075	 = Validation score   (f1_binary)
	103.32s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: NeuralNetTorch_r89_BAG_L1 ... Training model for up to 188873.36s of the 188873.36s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	0.2978	 = Validation score   (f1_binary)
	85.92s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 19308.87s of the 188785.06s of remaining time.
	Ensemble Weights: {'NeuralNetFastAI_r100_BAG_L1': 0.643, 'NeuralNetTorch_r197_BAG_L1': 0.339, 'NeuralNetFastAI_r191_BAG_L1': 0.018}
	0.3702	 = Validation score   (f1_binary)
	5.

In [6]:
print("AutoGluon infers problem type is:", predictor_ag.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor_ag.feature_metadata)

AutoGluon infers problem type is: binary
AutoGluon identified the following types of features:
('category', [])             : 6 | ['provincia', 'canton', 'distrito', 'estado_civil', 'Decil_Ingreso', ...]
('int', [])                  : 8 | ['num_propiedades', 'edad', 'IngEstimado_colones', 'IngresoPersonal', 'ingresoPadre', ...]
('int', ['bool'])            : 4 | ['genero', 'status_lab', 'jefehogar', 'Dummy_hijos']
('int', ['datetime_as_int']) : 3 | ['ANO_PERIODO', 'ANO_PERIODO.month', 'ANO_PERIODO.dayofweek']


In [7]:
predictor_ag.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                           model  score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0            WeightedEnsemble_L2   0.370224   f1_binary       0.317436   85.661507                0.006167           5.364306            2       True        111
1     NeuralNetTorch_r197_BAG_L1   0.363494   f1_binary       0.111897   31.541144                0.111897          31.541144            1       True         54
2    NeuralNetFastAI_r100_BAG_L1   0.357204   f1_binary       0.072262   15.945782                0.072262          15.945782            1       True        101
3     NeuralNetTorch_r185_BAG_L1   0.343643   f1_binary       0.128153   97.214866                0.128153          97.214866            1       True         78
4      NeuralNetTorch_r19_BAG_L1   0.339237   f1_binary       0.102546   55.698236                0.102546          55.698236        

/home/user/miniconda3/envs/ag_gpu/lib/python3.10/site-packages/autogluon/core/utils/plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'RandomForestGini_BAG_L1': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesGini_BAG_L1': 'StackerEnsembleModel_XT',
  'ExtraTreesEntr_BAG_L1': 'StackerEnsembleModel_XT',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'NeuralNetTorch_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_r177_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'NeuralNetTorch_r79_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBM_r131_BAG_L1': 'StackerEnsembleModel_LGB',
  'NeuralNetFastAI_r191_BAG_L1': 'StackerEnsembleModel_NNFastAiT

In [8]:
import time

t0 = time.time()
predictor_ag.feature_importance(df)
t1 = time.time()
print(t1-t0)
# 15 min

These features in provided data are not utilized by the predictor and will be ignored: ['MARCA', 'num_vehiculos', 'address']
Computing feature importance via permutation shuffling for 19 features using 5000 rows with 5 shuffle sets...
	117.89s	= Expected runtime (23.58s per shuffle set)
	74.56s	= Actual runtime (Completed 5 of 5 shuffle sets)


74.6949737071991


In [9]:
y_pred = predictor_ag.predict(df)

In [10]:
y_pred.value_counts()

target
0    4564
1    1652
Name: count, dtype: int64

In [11]:
df["target"]

school_profit(y_true=df["target"],y_pred=y_pred)

-72630

In [12]:
pd.DataFrame({"y_true" : df["target"],"y_pred": y_pred}).groupby(["y_true",	"y_pred"]).size()

y_true  y_pred
0       0         3997
        1          559
1       0          567
        1         1093
dtype: int64

In [13]:
4534*10 - 22*5 - 689*1

44541

In [16]:
df["target"].value_counts()

target
0    4556
1    1660
Name: count, dtype: int64

In [17]:
4556*10 - 1660

43900

In [14]:
school_profit(df["target"],y_pred,alpha=5,beta=10,gamma=1)

36608

In [18]:
1-36608/43900

0.16610478359908887

In [15]:
f1_score(y_true=df["target"],y_pred=y_pred, average="binary")

0.660024154589372